Data preprocessing
1. Make all files of same size
2. Combine them all in single image

Apply PCA followed by LDA
1. Data normalization
2. PCA
3. LDA

Test
1. Use above classifier to classify

PCA Algorithm
M = total images
1. For each training image
    1.1 convert it into single face N^2 * 1 vector
    1.2 subtract mean face from above vector
        1.2.1 Average mean face = 1/M(sum(all single face vector))
    1.3 Compute Covariance matrix
        1.3.1 A = set of all single face vector (A is N^2 * M matrix)
              C = 
              


In [1]:
# #Making same size of all images
# import numpy as np
# import os

# import cv2


# def resizee(filepath):
# 	#print filepath

# 	image = cv2.imread(filepath)
# 	# height, width, channels = image.shape

# 	resized = cv2.resize(image, (500,500), interpolation = cv2.INTER_AREA)
# 	# height, width, channels = resized.shape

# 	fp=os.getcwd()+"/ML facedata/"
# 	cv2.imwrite(fp+filename, resized)

# for filename in os.listdir(os.getcwd()+"/ML face images"):
# 	resizee("ML face images/"+filename)

In [2]:
#importing required library
import os
import cv2
import numpy as np

In [3]:
#initializing global parameters
folder_tr = "pencil/train"
folder_te = "pencil/test"
total_tr = 224
total_te = 82
dim = 1200
threshold = 90



In [4]:
#preparing lookup array
lookup = []
def fn(text):
    ans=[]
    for x in text:
        if x == '_':
            return (ans)
        elif x== 'O':
            return (ans)
        elif x == 'o':
            return (ans)
        else:
            ans+=x
folder="cc"
for filename in os.listdir(os.getcwd()+"/"+folder_tr):
    ans = fn(filename)
    ans = ''.join(str(e) for e in ans)
    lookup.append(ans)


len(lookup)

224

In [5]:
#L is set of all training images
#Currently L is zero matrix
import numpy as np
L = np.empty(shape=(dim*dim,total_tr), dtype='float64')
print(L.shape)
i = 0
import os,cv2

for filename in os.listdir(os.getcwd()+"/"+folder_tr):
    #Read a image 
    image = cv2.imread(folder_tr+"/"+filename)
    resized = cv2.resize(image, (dim,dim), interpolation = cv2.INTER_AREA)
    #lookup.append(filename)
    #Convert it into grayscale image
    gray_image = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
    #flattening an image
    
    gray_vector = np.array(gray_image, dtype='float64').flatten()
    L[:, i] = gray_vector[:] 
    #temp.append(gray_vector)
    i = i + 1

print("done")

(1440000, 224)
done


(700, 523, 3)

In [6]:
#Mean face
mean_face = np.sum(L, axis=1) /total_tr

# # mean_face = np.matrix(mean_face)
# # mean_face = mean_face.T
mean_face[:]

#subtract mean face
for i in range(total_tr):
    L[:,i] -= mean_face[:]

print("L",L.shape)
print("LT",L.T.shape)
print("mean face",mean_face.shape)

LT = L.transpose()

#calculate LTL: It is covariance matrix
C = np.matrix(LT) * np.matrix(L)  
C = C/total_tr
print("Covariance (LT)(L)",C.shape)

#calculate eigenvector and eigenvalue of covariance matrix
evalues,evectors = np.linalg.eig(C)

#getting correct ordering
sort_indices = evalues.real.argsort()[::-1]

evalues = evalues.real[sort_indices]
evectors = evectors[sort_indices]

#Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(evalues[i], evectors[:,i]) for i in range(len(evalues))]
print("done")

L (1440000, 224)
LT (224, 1440000)
mean face (1440000,)
Covariance (LT)(L) (224, 224)
done


In [7]:
#decide value of K 
import plotly
import plotly.plotly as py
from plotly.graph_objs import *
import plotly.tools as tls

tot = sum(evalues)
var_exp = [(i / tot)*100 for i in sorted(evalues, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

K = 0
#print(abs(cum_var_exp))
for i in range(len(cum_var_exp)):
    K = i
    if(int(cum_var_exp[i])==threshold):
        break
print(K)
trace1 = Bar(
        x=['PC %s' %i for i in range(1,total_tr)],
        y=var_exp,
        showlegend=False)

trace2 = Scatter(
        x=['PC %s' %i for i in range(1,total_tr)], 
        y=cum_var_exp,
        name='cumulative explained variance')

data = Data([trace1, trace2])

layout=Layout(
        yaxis=YAxis(title='Explained variance in percent'),
        title='Explained variance by different principal components')

fig = Figure(data=data, layout=layout)
#plotly.offline.plot(fig)
print("done")

89
done


In [8]:
#taking K eigenvector and putting into cols of P
P = eig_pairs[0][1]
for j in range(K-1):    
    b = eig_pairs[j+1][1]
    P = np.hstack((P,b)) 
print("done")

done


In [9]:
#projecting
evectors = L * P 

# #normalize eigenvectors
norms = np.linalg.norm(evectors, axis=0)
norms.shape
evectors = evectors / norms 
# #computing weights
W = evectors.transpose() * L  
W.shape
print("done")

done


In [10]:
#classification
test_no = []
i = 0
wrong = 0
for filename in os.listdir(os.getcwd()+"/"+folder_te):
    ans = fn(filename)
    ans = ''.join(str(e) for e in ans)
    test_no.append(ans)
    #print("i ",i,"test_no ",test_no[0])
    test = cv2.imread(folder_te+"/"+filename)    # read as a grayscale image
    test = cv2.resize(test, (dim,dim), interpolation = cv2.INTER_AREA)
    test= cv2.cvtColor(test, cv2.COLOR_BGR2GRAY)
    img_col = np.array(test, dtype='float64').flatten()  
    #print(img_col.shape)
    img_col -= mean_face                                           # subract the mean column
    img_col = np.reshape(img_col, (dim*dim, 1))                             # from row vector to col vector

    S = evectors.transpose() * img_col                                 # projecting the normalized probe onto the
                                                                            # Eigenspace, to find out the weights

    diff = W - S                                                       # finding the min ||W_j - S||
    norms = np.linalg.norm(diff, axis=0)

    closest_face_id = np.argmin(norms)    
    #print(test_no[i]+" and "+lookup[closest_face_id])
    if(test_no[i]!=lookup[closest_face_id]):
        wrong = wrong + 1
    i = i + 1

print("done")

done


In [11]:
#Accuracy
print(wrong)
1 - wrong/total_te

32


0.6097560975609756

In [12]:
# #making N*N matrix for each from N^2 * 1 image
# import matplotlib.pyplot as plt
# for i in range(K):
#     gray_vector1 = np.reshape(Y.T[i], (1000, 1000))
#     #plt.imshow(gray_vector1,cmap='gray')
    #plt.show()